In [ ]:
%config Completer.use_jedi = False

In [ ]:
import sys, os
sys.path.append("/home/files/cnsi-sed2")

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0, EfficientNetB7
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, ReLU
import tensorflow.keras.layers as layers
import tensorflow as tf
import numpy as np
import os

def resize_images(images, size, method='bilinear', align_corners=False):
    """ See https://www.tensorflow.org/versions/r1.14/api_docs/python/tf/image/resize_images .
    Args
        method: The method used for interpolation. One of ('bilinear', 'nearest', 'bicubic', 'area').
    """
    methods = {
        'bilinear': tf.image.ResizeMethod.BILINEAR,
        'nearest' : tf.image.ResizeMethod.NEAREST_NEIGHBOR,
        'bicubic' : tf.image.ResizeMethod.BICUBIC,
        'area'    : tf.image.ResizeMethod.AREA,
    }
    return tf.compat.v1.image.resize_images(images, size, methods[method], align_corners)

class UpsampleLike(tf.keras.layers.Layer):
    """ Keras layer for upsampling a Tensor to be the same shape as another Tensor.
    """

    def call(self, inputs, **kwargs):
        source, target = inputs
        target_shape = tf.keras.backend.shape(target)
        if tf.keras.backend.image_data_format() == 'channels_first':
            source = tf.transpose(source, (0, 2, 3, 1))
            output = resize_images(source, (target_shape[2], target_shape[3]), method='nearest')
            output = tf.transpose(output, (0, 3, 1, 2))
            return output
        else:
            return resize_images(source, (target_shape[1], target_shape[2]), method='nearest')

    def compute_output_shape(self, input_shape):
        if tf.keras.backend.image_data_format() == 'channels_first':
            return (input_shape[0][0], input_shape[0][1]) + input_shape[1][2:4]
        else:
            return (input_shape[0][0],) + input_shape[1][1:3] + (input_shape[0][-1],)


# The number of anchors

In [ ]:
from utils import anchor

anchor_param = {"ratios": [0.5, 1, 2],
                "scales": [1.0, 1.25,1.58],
                           "fm_sizes": [64, 32, 16, 8, 4],
                           "image_size": 512} #anchor parameters
default_boxes = anchor.generate_retina_boxes(anchor_param)

In [ ]:
default_boxes

# Efficient + RetinaNet build

In [ ]:
input_layer = tf.keras.Input(shape=(512,512,3))
backbone = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=input_layer)

In [ ]:
#tf.keras.utils.plot_model(backbone, to_file='model.png', show_shapes=True, dpi=96)

In [ ]:
C1 = backbone.get_layer("block1a_project_bn").output
print("C1", C1.shape)
C2 = backbone.get_layer("block2b_add").output
print("C2", C2.shape)
C3 = backbone.get_layer("block3b_add").output
print("C3", C3.shape)
C4 = backbone.get_layer("block4c_add").output
print("C4", C4.shape)
C5 = backbone.get_layer("block5c_add").output
print("C5", C5.shape)
C6 = backbone.get_layer("block6d_add").output
print("C6", C6.shape)
C7 = backbone.get_layer("top_activation").output
print("C7", C7.shape)

In [ ]:
print(C3.shape, C5.shape, C7.shape) # They will be P3,P4,P5

In [ ]:
def FPN(features, channel_depth):
    P3,P4,P5 = features   
    P5 = Conv2D(channel_depth,kernel_size=1,strides=1,padding='same')(P5)
    P5_upsampled = UpsampleLike(name='P5_upsampled')([P5, P4])
    
    P4 = Conv2D(channel_depth,kernel_size=1,strides=1,padding='same')(P4)
    P4 = tf.add(P4, P5_upsampled)
    P4_upsampled = UpsampleLike(name='P4_upsampled')([P4, P3])
    
    P3 = Conv2D(channel_depth,kernel_size=1,strides=1,padding='same')(P3)
    P3 = tf.add(P3, P4_upsampled)
    
    P6 = Conv2D(channel_depth,kernel_size=3,strides=2,padding='same')(P5)
    P7 = Conv2D(channel_depth,kernel_size=3,strides=2,padding='same')(P6)
    P7 = ReLU()(P7)
    return [P3,P4,P5,P6,P7] 

[P3,P4,P5,P6,P7] = FPN([C3,C5,C7], 256)
print(P3.shape,P4.shape,P5.shape,P6.shape,P7.shape)

In [ ]:
channel_depth = 256
num_classes = 4
k_anchors = 15

def classification_head(_input, channel_depth, fmap_size, num_classes, num_anchors):
    x = Conv2D(channel_depth, kernel_size=3, padding='same')(_input)
    x = ReLU()(x)
    x = Conv2D(channel_depth, kernel_size=3, padding='same')(x)
    x = ReLU()(x)
    x = Conv2D(channel_depth, kernel_size=3, padding='same')(x)
    x = ReLU()(x)
    x = Conv2D(channel_depth, kernel_size=3, padding='same')(x)
    x = ReLU()(x)
    x = Conv2D(num_classes*num_anchors, kernel_size=3, padding='same')(x)
    x = tf.keras.activations.sigmoid(x)
    x = tf.reshape(x, [-1, fmap_size*fmap_size*num_anchors,num_classes])
    return x
    
#conf header
confs = []
fmap_size = 64
conf = classification_head(P3, channel_depth, fmap_size, num_classes, k_anchors)
confs.append(conf)
    
fmap_size = 32
conf = classification_head(P4, channel_depth, fmap_size, num_classes, k_anchors)
confs.append(conf)

fmap_size = 16
conf = classification_head(P5, channel_depth, fmap_size, num_classes, k_anchors)
confs.append(conf)

fmap_size = 8
conf = classification_head(P6, channel_depth, fmap_size, num_classes, k_anchors)
confs.append(conf)

fmap_size = 4
conf = classification_head(P7, channel_depth, fmap_size, num_classes, k_anchors)
confs.append(conf)

confs = tf.concat(confs, axis=-2)
print(confs.shape)

In [ ]:
channel_depth = 256
coords = 4
k_anchors = 15

def bbox_regression_head(_input, channel_depth, fmap_size, coords, num_anchors):
    x = Conv2D(channel_depth, kernel_size=3, padding='same')(_input)
    x = ReLU()(x)
    x = Conv2D(channel_depth, kernel_size=3, padding='same')(x)
    x = ReLU()(x)
    x = Conv2D(channel_depth, kernel_size=3, padding='same')(x)
    x = ReLU()(x)
    x = Conv2D(channel_depth, kernel_size=3, padding='same')(x)
    x = ReLU()(x)
    x = Conv2D(coords*num_anchors, kernel_size=3, padding='same')(x)
    x = tf.reshape(x, [-1, fmap_size*fmap_size*num_anchors,coords])
    return x
    
#conf header
locs = []
fmap_size = 64
loc = bbox_regression_head(P3, channel_depth, fmap_size, coords, k_anchors)
locs.append(loc)
    
fmap_size = 32
loc = bbox_regression_head(P4, channel_depth, fmap_size, coords, k_anchors)
locs.append(loc)

fmap_size = 16
loc = bbox_regression_head(P5, channel_depth, fmap_size, coords, k_anchors)
locs.append(loc)

fmap_size = 8
loc = bbox_regression_head(P6, channel_depth, fmap_size, coords, k_anchors)
locs.append(loc)

fmap_size = 4
loc = bbox_regression_head(P7, channel_depth, fmap_size, coords, k_anchors)
locs.append(loc)

locs = tf.concat(locs, axis=-2)
print(locs.shape)

In [ ]:
Retinanet = tf.keras.Model(inputs=input_layer, outputs=[confs, locs])

In [ ]:
Retinanet.summary()

In [ ]:
tf.keras.utils.plot_model(Retinanet)

In [ ]:
Retinanet.save("Retina-efficient-512-imagenet.h5")